## 扱う質問

ベースモデルでは答えられない（最近起きたこと）ものとして最初に「最近、大谷翔平に子供が出来た」を思いついたので、大谷翔平についての新旧の質問をしてみる。（ちょうど大リーグ中継を見ていたため）

## 扱うモデル

Meta-Llama-3-8B-Instruct（2024年4月リリース）を使用します。演習のサンプルそのままです。

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
# !pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# 演習用のコンテンツを取得（forkした自分のレポジトリ）
!git clone https://github.com/MasayukiK/lecture-ai-engineering

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 81 (delta 6), reused 14 (delta 6), pack-reused 64 (from 2)
Receiving objects: 100% (81/81), 442.88 KiB | 1.77 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
messages = [
    {"role": "system", "content": "質問に全て回答してください。必ず「日本語で回答」すること。"},
    {"role": "user", "content": "[質問1]大谷翔平の子供の性別は？"},
    {"role": "user", "content": "[質問2]大谷翔平の兄弟は誰？"},
    {"role": "user", "content": "[質問3]大谷翔平の妻は誰？"},
    {"role": "user", "content": "[質問4]大谷翔平の所属球団は？"},
    {"role": "user", "content": "[質問5]大谷翔平の出身高校は？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Here are the answers to your questions about Shohei Ohtani:

[質問1]大谷翔平の子供の性別は？
Shohei Ohtani does not have any children.

[質問2]大谷翔平の兄弟は誰？
Shohei Ohtani has an older brother named Shingo Ohtani.

[質問3]大谷翔平の妻は誰？
Shohei Ohtani is not married.

[質問4]大谷翔平の所属球団は？
Shohei Ohtani currently plays for the Los Angeles Angels in Major League Baseball (MLB).

[質問5]大谷翔平の出身高校は？
Shohei Ohtani attended Hanamigawa High School in Hokkaido, Japan.


## 結果 (ベースモデル)

Meta-Llama-3-8B-Instructは「大谷翔平」について誤った知識を提示しました。そして何故か指示に従わず英語で回答している...

---

# 2. 文字起こしデータの活用
## 手作業で用意したテキストデータをソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: WikiPedia の大谷翔平のページ。（家族について記載した部分のみ抽出）<br>
ダサいが、上記からのコピペでテキストファイルを作成しただけ。ちゃんと自動化したいところ。

* **目的**: モデルに「大谷翔平」の家族に関する最新の知識を提供し、事実に基づいた回答を促す

In [ ]:
with open("/content/lecture-ai-engineering/day3_me/data/OtaniSyouhei.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:

references = raw_writedown
messages = [
    {"role": "system", "content": "質問に全て回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": "[質問1]大谷翔平の子供の性別は？"},
    {"role": "user", "content": "[質問2]大谷翔平の兄弟は誰？"},
    {"role": "user", "content": "[質問3]大谷翔平の妻は誰？"},
    {"role": "user", "content": "[質問4]大谷翔平の所属球団は？"},
    {"role": "user", "content": "[質問5]大谷翔平の出身高校は？"},
    {"role": "user", "content": f"\n\n[参考資料]\n{references}"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

[回答]

[質問1]大谷翔平の子供の性別は、長女。

[質問2]大谷翔平の兄弟は、7歳上の実兄・大谷龍太と2歳上の姉。

[質問3]大谷翔平の妻は、田中真美子。

[質問4]大谷翔平の所属球団は、現在の所属球団は不明（参考資料には球団名が記載されていない）。

[質問5]大谷翔平の出身高校は、岩手県立黒沢尻工業高等学校（父親の大谷徹が卒業した高校）。


---------------------------------------------------------



## 結果の考察
与えたデータに答えが存在しない質問については正解できていない。
ベースモデルそのままより悪化しているものもある。
出身高校についてはハルシネーションを起こしている。





# 3. もう一度トライ　**（未完）**

ここまでのは単純すぎたので（初心者の練習ということで...）、huggingface のデータセットを利用してみる。
事前に中にどのような情報が入っているか確認してから質問を考えなければならない。

In [ ]:
#from huggingface_hub import login
#login()

In [ ]:
# RAG データ
!pip install datasets
from datasets import load_dataset

ds = load_dataset("allganize/RAG-Evaluation-Dataset-JA")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


README.md:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

rag_evaluation_result.csv:   0%|          | 0.00/5.73M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]